# GCN
Build the model with the following structure using the PyTorch Geometric library:
1. Input layer
2. 1st hidden layer: GCNConv + ReLU
3. 2nd hidden layer: GCNConv + ReLU
4. 3rd hidden layer: GCNConv + global mean pooling + Dropout
5. Output layer: Linear
* Loss function: CrossEntropyLoss
* Optimizer: Adam

**WARNING** <br>
Execute only colab


In [1]:
# install torch-geometric and related libraries
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=0f60c640f09f05348e3f2558129086d6af7fc4e9ce046bcc4cb4bc3b33973be4
  Stored in directory: /root/.cache/pip/wheels/31/b2/8c/9b4bb72a4384eabd1ffeab2b7ead692c9165e35711f8a9dc72
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.p

## Load the dataset

In [5]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

# load dataset
dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG')

# split dataset into train, validation and test
dataset = dataset.shuffle()
train_dataset = dataset[:140]
val_dataset = dataset[140:]

# create dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## Build the model

In [6]:
# define model
# 1. Input layer
# 2. 1st hidden layer: GCNConv + ReLU
# 3. 2nd hidden layer: GCNConv + ReLU
# 4. 3rd hidden layer: GCNConv + global mean pooling + Dropout
# 5. Output layer: Linear

import torch
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import GCNConv, global_mean_pool

n_h = 64

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, n_h)
        self.conv2 = GCNConv(n_h, n_h)
        self.conv3 = GCNConv(n_h, n_h)
        self.fc = Linear(n_h, dataset.num_classes)

        self.relu = ReLU()
        self.dropout = Dropout(p=0.5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.relu(x)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.fc(x)
        return x

net = Net()
net.cuda() # use GPU

Net(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (fc): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

# Train the model

In [7]:
# define loss function and optimizer
# * Loss function: CrossEntropyLoss
# * Optimizer: Adam
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

criterion = CrossEntropyLoss()
optimizer = Adam(net.parameters())

# define evaluation function
def evaluate(loader):
    correct = 0
    for data in loader:
        data = data.cuda() # use GPU
        out = net(data)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)  * 100

# train model
for epoch in range(200):
    net.train()
    for data in train_loader:
        data = data.cuda() # use GPU
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

    # evaluate model
    net.eval()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.2f}, Val Acc: {val_acc:.2f}')

Epoch: 000, Train Acc: 65.71, Val Acc: 68.75
Epoch: 001, Train Acc: 65.71, Val Acc: 68.75
Epoch: 002, Train Acc: 65.71, Val Acc: 68.75
Epoch: 003, Train Acc: 65.71, Val Acc: 68.75
Epoch: 004, Train Acc: 65.71, Val Acc: 68.75
Epoch: 005, Train Acc: 65.71, Val Acc: 68.75
Epoch: 006, Train Acc: 65.71, Val Acc: 68.75
Epoch: 007, Train Acc: 65.71, Val Acc: 68.75
Epoch: 008, Train Acc: 65.71, Val Acc: 68.75
Epoch: 009, Train Acc: 65.71, Val Acc: 68.75
Epoch: 010, Train Acc: 65.71, Val Acc: 68.75
Epoch: 011, Train Acc: 65.71, Val Acc: 68.75
Epoch: 012, Train Acc: 65.71, Val Acc: 68.75
Epoch: 013, Train Acc: 65.71, Val Acc: 68.75
Epoch: 014, Train Acc: 65.71, Val Acc: 68.75
Epoch: 015, Train Acc: 65.71, Val Acc: 68.75
Epoch: 016, Train Acc: 65.71, Val Acc: 68.75
Epoch: 017, Train Acc: 65.71, Val Acc: 68.75
Epoch: 018, Train Acc: 65.71, Val Acc: 68.75
Epoch: 019, Train Acc: 65.71, Val Acc: 68.75
Epoch: 020, Train Acc: 65.71, Val Acc: 68.75
Epoch: 021, Train Acc: 65.71, Val Acc: 68.75
Epoch: 022

## Evaluate the model

In [8]:
net.eval()
test_acc = evaluate(val_loader)
print(f'Test Acc: {test_acc:.2f}')

Test Acc: 89.58
